## **Importar las librerias**

In [1]:
import numpy as pd
import pandas as pd
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

## **Se crea un Dataframe con el archivo csv**

In [2]:
df = pd.read_csv('dfListo.csv')
df.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,id_collection,name_collection,id_genres,name_genres,id_production,name_production,id_language,name_language,id_countrie,name_countrie
0,30000000.0,862.0,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,10194.0,Toy Story Collection,"16.0,35.0,10751.0","Animation,Comedy,Family",3.0,Pixar Animation Studios,en,English,US,United States of America
1,65000000.0,8844.0,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,...,NaN,NaN,"12.0,14.0,10751.0","Adventure,Fantasy,Family","559.0,2550.0,10201.0","TriStar Pictures,Teitler Film,Interscope Commu...","en,fr","English,Français",US,United States of America
2,0.0,15602.0,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,...,119050.0,Grumpy Old Men Collection,"10749.0,35.0","Romance,Comedy","6194.0,19464.0","Warner Bros.,Lancaster Gate",en,English,US,United States of America
3,16000000.0,31357.0,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,...,NaN,NaN,"35.0,18.0,10749.0","Comedy,Drama,Romance",306.0,Twentieth Century Fox Film Corporation,en,English,US,United States of America
4,0.0,11862.0,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,...,96871.0,Father of the Bride Collection,35.0,Comedy,"5842.0,9195.0","Sandollar Productions,Touchstone Pictures",en,English,US,United States of America


## **Se limita el dataframe**

In [3]:
ejemplo=df.sample(n=1000, random_state=42)

## **Desarrollo del modelo**

In [4]:
tfidf = TfidfVectorizer(stop_words='english')

# Rellenar los valores nulos en 'overview' con espacios vacíos
ejemplo['overview'] = ejemplo['overview'].fillna('')

# Entrenamiento del modelo TfidfVectorizer
tfidf_matrix = tfidf.fit_transform(ejemplo['overview'])

# Cálculo de similitud del coseno
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Eliminación de duplicados en ml
ejemplo.drop_duplicates(inplace=True)
ejemplo = ejemplo.reset_index(drop=True)

# Creación de la serie de índices con los títulos de las películas
indices = pd.Series(ejemplo.index, index=ejemplo['title'])

In [5]:
tfidf_matrix.shape # te arroja el resultado

(1000, 10120)

## Se realizan las siguientes funciones

In [6]:
def recomendaciones(title, cosine_sim=cosine_sim):
    # Obtener el índice correspondiente al título
    idx = indices[title]
    
    # Obtener los puntajes de similitud y enumerarlos
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Ordenar los puntajes de similitud de forma descendente
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Obtener los 5 elementos más similares (excluyendo el título actual)
    sim_scores = sim_scores[1:6]
    
    # Obtener los índices de las películas recomendadas
    movie_indices = [i[0] for i in sim_scores]
    
    # Devolver los títulos de las películas recomendadas
    return ejemplo['title'].iloc[movie_indices]

## **Con el fin de probar el modelo tenemos que verificar que titulos podemos usar como input**

In [7]:
ejemplo['title'].head()

0        I Am Belfast
1       Office Killer
2        Lovely Molly
3            Blockade
4    Statues Also Die
Name: title, dtype: object

In [8]:
recomendaciones('Blockade')

274              Spicebush
727              10,000 km
717       American Pastime
340        Miss Rose White
458    No Love for Johnnie
Name: title, dtype: object